In [5]:
import reasoning_gym

In [6]:
with open("data.txt") as f:
    data_names = f.readlines()
    data_names = [name.strip() for name in data_names]
    print("Total number of data: ", len(data_names))

Total number of data:  56


In [7]:
TOTAL = 10000
collisions = []

In [8]:
%%time
for name in data_names:
    data_1 = reasoning_gym.create_dataset(name, size=TOTAL, seed=1)
    data_2 = reasoning_gym.create_dataset(name, size=TOTAL, seed=2)
    count = 0
    for item_1, item_2 in zip(data_1, data_2):
        if item_1["question"] == item_2["question"]:
            count += 1

    # Add name, count to collisions.txt
    with open('collisions_1.txt', 'a') as file:
        file.write(f"{name}, {count}\n")

SyntaxError: invalid syntax (<string>, line 1)

In [9]:
data_1 = reasoning_gym.create_dataset("basic_arithmetic", size=TOTAL, seed=1)

In [10]:
len(data_1)

10000

In [12]:
data_2 = reasoning_gym.create_dataset("basic_arithmetic", size=TOTAL, seed=2)

In [13]:
len(data_2)

10000

In [14]:
data_1[105]

{'question': 'Calculate -( -1 + ( 0 * -2 ) / 3 + 6 ).',
 'answer': '-5',
 'metadata': {'source_dataset': 'basic_arithmetic',
  'source_index': 105,
  'expression': '-( -1 + ( 0 * -2 ) / 3 + 6 )',
  'num_terms': 5,
  'num_digits': 1,
  'difficulty': {'num_terms': (2, 6), 'num_digits': (1, 4)}}}

In [15]:
data_2[105]

{'question': 'Calculate -( -( -8861 * -813 ) ) * -3862.',
 'answer': '-27821820966',
 'metadata': {'source_dataset': 'basic_arithmetic',
  'source_index': 105,
  'expression': '-( -( -8861 * -813 ) ) * -3862',
  'num_terms': 3,
  'num_digits': 4,
  'difficulty': {'num_terms': (2, 6), 'num_digits': (1, 4)}}}